In [7]:
import openai
import pandas as pd

API_KEY = "YOUR_API_KEY"

In [8]:
df = pd.read_csv("../data/cleanedTweets.csv") 

# prompt = '''The following tweets are related to ethereum price variation, where people can face a bulish or a bearish attitude/sentiment of them. Each time i'll sendo you 5 tweets, so each prompt needs to have as response an array with 5 values, each one corresponding to each tweet.
# Do a sentiment analysis of the tweets, by returning ONLY an array with the value for each one. 
# Note that the values should range from -1 to 1, meaning that -1 is very negative and 1 is very positive, and return JUST the array:

# '''

# prompt = '''The following tweets are related to ethereum price variation, where people can face a bulish or a bearish attitude/sentiment of them. 
# Do a sentiment analysis of the tweets, by returning ONLY an array with the value for each one. 
# Note that the values should range from -1 to 1, meaning that -1 is very negative and 1 is very positive, and return JUST the array:

# '''

promptSingleTweet = '''The following tweet is related to ethereum price variation, where people can face a bulish or a bearish attitude/sentiment of them.
Do a sentiment analysis of the tweet, by returning ONLY the value for the tweet.  Remember the whole phrase is just one tweet, which you will only give me the value of the sentiment analysis.
Note that the values should range from -1 to 1, meaning that -1 is very negative and 1 is very positive, and return JUST the value of this tweet:
'''

#### Extrair um tweet de cada vez, mas com a divisão do dataset em 4, para que se minimize as perdas com erros

In [12]:
import time
import openai
from requests.exceptions import ConnectionError
import re
import numpy as np

def testeModel(df, prompt):
    openai.api_key = API_KEY

    compound_values = []  # Lista para armazenar os valores de sentimentos

    for index, row in df.iterrows():
        tweet = row["Tweet"]
        prompt_line = prompt + tweet
        print(prompt_line)

        retries = 3
        while retries > 0:
            try:
                response = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=prompt_line,
                    temperature=0,
                    max_tokens=256,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0
                )
                break
            except (ConnectionError, openai.OpenAIError) as e:
                print(f"Error communicating with OpenAI: {str(e)}. Retrying...")
                retries -= 1
                time.sleep(2)  # Wait for a few seconds before retrying

        if retries == 0:
            print("Max retries reached. Skipping request.")
            continue

        print(response)

        output_text = response["choices"][0]["text"]
        sentiment_match = re.search(r"[-+]?\d+(\.\d+)?", output_text)
        if sentiment_match:
            compound_value = float(sentiment_match.group(0))
            print(compound_value)
            # Limita o valor entre -1 e 1, caso o chatbot retorne um valor fora desse intervalo
            if compound_value > 1:
                compound_value = 1
            elif compound_value < -1:
                compound_value = -1

            compound_values.append(compound_value)  # Adiciona o valor à lista

            df.loc[index, "compoundChatGPT"] = compound_value

    return df, compound_values


num_parts = 4
df_parts = np.array_split(df, num_parts)

compound_values_all = []

for i, df_part in enumerate(df_parts):
    df_res, compound_values = testeModel(df_part, promptSingleTweet)
    df_res.to_csv(f"../data/openAI/openAIResults_{i}.csv", index=False)
    compound_values_all.extend(compound_values)

print(compound_values_all)


The following tweet is related to ethereum price variation, where people can face a bulish or a bearish attitude/sentiment of them.
Do a sentiment analysis of the tweet, by returning ONLY the value for the tweet.  Remember the whole phrase is just one tweet, which you will only give me the value of the sentiment analysis.
Note that the values should range from -1 to 1, meaning that -1 is very negative and 1 is very positive, and return JUST the value of this tweet:
happy friday fam   look for the change in the picture   friday is always the best, enjoy your day!   stay active, stay bullish. we are on the way  glad to have you   nftcommunity nft nfts eth ethereum web3 nftart bayc dubaibornapes dubai 
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n0.8"
    }
  ],
  "created": 1686751511,
  "id": "cmpl-7RLPTRNucSeOdVvPOB5JuxACy9xuN",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_

In [13]:
import pandas as pd

num_parts = 4
df_combined = pd.DataFrame()  # DataFrame vazio para armazenar os datasets combinados

for i in range(num_parts):
    df_part = pd.read_csv(f"../data/openAI/openAIResults_{i}.csv")
    df_combined = pd.concat([df_combined, df_part])

df_combined.to_csv("../data/openAI/openAIResults_combined.csv", index=False)
